## Fine-tuned HuBERT model
This notebook trains a model to predict the label of a vocalization by individual `P05`. Labels with fewer than 30 examples in the training data are discarded. Remaining labels:
 - `selftalk` (231 examples)
 - `frustrated` (223 examples)
 - `delighted` (182 examples)
 - `dysregulated` (92 examples) 
 - `happy` (49 examples)


I testing using ordinary 10-fold CV, and also leave-one-session-out CV. Performance is significantly better with 10-fold CV, which indicates that the model is picking up on things specific to sessions (perhaps background noise). I expect leave-one-session-out CV is a more accurate estimation of the generalizability of a model.


In [1]:
import functools
from pathlib import Path


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    log_loss,
    recall_score,
)
from sklearn.model_selection import (
    cross_val_predict,
    StratifiedKFold,
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import torch
import torchaudio
from tqdm.notebook import tqdm


def to_prob(metric):
    @functools.wraps(metric)
    def metric_that_takes_prob(y_actual, y_pred, sample_weight=None):
        return metric(y_actual, y_pred.argmax(1), sample_weight=sample_weight)

    return metric_that_takes_prob


metrics = {
    "accuracy": to_prob(accuracy_score),
    "balanced_accuracy": to_prob(balanced_accuracy_score),
    "unweighted_f1": to_prob(functools.partial(f1_score, average="macro")),
    "UAR": to_prob(functools.partial(recall_score, average="macro")),
    "logloss": log_loss,
}

In [2]:
# Pretrained HuBERT model
bundle = torchaudio.pipelines.HUBERT_BASE
model = bundle.get_model()

# List of data files
data_files = pd.read_csv("../data/directory_w_train_test.csv")
data_files_p5 = data_files.loc[data_files.Participant == "P05"]
label_counts = data_files_p5.Label.value_counts()
training_files = data_files_p5.loc[
    data_files_p5.Label.isin(label_counts[label_counts >= 30].index)
    & (data_files_p5.is_test == 0)
].copy()
training_files["session"] = training_files.Filename.apply(
    lambda name: name.split("-")[0][:-3]
)
display(training_files.Label.value_counts())

Label
selftalk        231
frustrated      223
delighted       182
dysregulated     92
happy            49
Name: count, dtype: int64

In [3]:
# Extract acoustic features from one wav file using
# the pretrained

datadir = Path("../data/wav")
filename = training_files.Filename.iloc[0]
waveform, sample_rate = torchaudio.load(datadir / filename)
waveform = torchaudio.functional.resample(
    waveform, sample_rate, bundle.sample_rate
)
features, _ = model.extract_features(waveform)

# features is a list of 12 tensors, each having shape
# (m, n, 768), where the value of m and n are different
# depending on the audio sample (m is always 1 or 2, n
# varies more widely and I think depends on the length
# of the clip). I'm averaging over time (n).
print(f"{type(features)=}, {len(features)=}")
print(features[0].mean((0, 1)).shape)

type(features)=<class 'list'>, len(features)=12
torch.Size([768])


In [4]:
# Generate features using the pretrained model.
# We will use only the first layer of generated features.
with torch.no_grad():
    t_list = []
    for filename in tqdm(training_files.Filename):
        waveform, sample_rate = torchaudio.load(datadir / filename)
        waveform = torchaudio.functional.resample(
            waveform, sample_rate, bundle.sample_rate
        )

        features, _ = model.extract_features(waveform)
        t_list.append(features[0].mean((0, 1)))

X = torch.stack(t_list).detach()
labels = training_files.Label.unique()
y = torch.zeros(len(training_files), dtype=torch.int)
for idx, label in enumerate(labels):
    y[(training_files.Label == label).values] = idx
print(X.shape, y.shape)

  0%|          | 0/777 [00:00<?, ?it/s]

torch.Size([777, 768]) torch.Size([777])


In [5]:
# There are 768 generated features, which is a lot
# relative to how many training data there are. So we
# will need regularization. Using sk-optimize to optimize
# strength of regularization parameter (this is overkill
# since there's just one parameter, but oh well)
est = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        max_iter=10**6,
    ),
)
opt = BayesSearchCV(
    est,
    {
        "logisticregression__C": (5e-3, 1, "log-uniform"),
    },
    n_iter=20,
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=12345),
    scoring="accuracy",
)
opt.fit(
    X.reshape(len(X), -1),
    y,
)
print(opt.best_params_)
print("Best accuracy:", opt.best_score_)

OrderedDict([('logisticregression__C', 0.01331684278118987)])
Best accuracy: 0.6654345654345655


In [6]:
# Some possible sample weights for training and/or metrics.
# session_weight weights each sample based on sessions, so
# that the total weight of observations in each sesion is
# constant. On top of that, session_and_label_weight assigns
# a label to each weight, which is multiplied by the sesion weight,
# in such a way to make the sum of the weights constant by label.
session_weight = (
    (1 / training_files.session.value_counts())
    .clip(None, 0.1)
    .loc[training_files.session]
).values
session_and_label_weight = (
    1
    / pd.Series(session_weight, training_files.index)
    .groupby(training_files.Label)
    .sum()
).loc[training_files.Label].values * session_weight

In [7]:
# Generate out-of-sample predictions using a logistic
# regression model, with the parameter determined by
# the optimization above.
#
# We compute various metrics, using various weightings
est = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        C=opt.best_params_["logisticregression__C"],
        max_iter=10**6,
    ),
)
oos_pred_prob = cross_val_predict(
    est,
    X.reshape(len(X), -1),
    y,
    cv=StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=1234,  # Using different seed to avoid over-fitting parameter
    ),
    method="predict_proba",
    params={"logisticregression__sample_weight": session_weight},
)
oos_pred = oos_pred_prob.argmax(1)

display(
    pd.Series(
        {name: metric(y, oos_pred_prob) for name, metric in metrics.items()},
        name="no_weight",
    ).round(3)
)
display(
    pd.Series(
        {
            name: metric(y, oos_pred_prob, sample_weight=session_weight)
            for name, metric in metrics.items()
        },
        name="session_weight",
    ).round(3)
)
display(
    pd.Series(
        {
            name: metric(
                y, oos_pred_prob, sample_weight=session_and_label_weight
            )
            for name, metric in metrics.items()
        },
        name="session_and_label_weight",
    ).round(3)
)

accuracy             0.614
balanced_accuracy    0.511
unweighted_f1        0.525
UAR                  0.511
logloss              1.066
Name: no_weight, dtype: float64

accuracy             0.645
balanced_accuracy    0.528
unweighted_f1        0.549
UAR                  0.528
logloss              1.028
Name: session_weight, dtype: float64

accuracy             0.528
balanced_accuracy    0.528
unweighted_f1        0.496
UAR                  0.528
logloss              1.239
Name: session_and_label_weight, dtype: float64

In [8]:
# Confusion matrix
# selftalk and delighted are frequently confused by this model
conf_matrix_df = pd.DataFrame(
    confusion_matrix(y, oos_pred), columns=labels, index=labels
)
conf_matrix_df.index.name = "actual_label"
conf_matrix_df.columns.name = "pred_label"
display(conf_matrix_df)

pred_label,happy,frustrated,dysregulated,selftalk,delighted
actual_label,,,,,
happy,17,1,1,15,15
frustrated,0,163,5,19,36
dysregulated,4,32,11,33,12
selftalk,0,7,0,185,39
delighted,0,10,2,69,101


In [9]:
# Hold one session out CV
# Performance is substantially worse, so the model
# is probably picking up on background sounds. But
# there is still some predictive power.

# Here I'm running the CV for many different values
# of the regularization parameter C, and logging
# some different scores. Optimal value of C depends
# on whether whether metrics weight classes by class
# size.
scores = []

# C_list = [0.02 * 0.8**n for n in range(20)]
# for C in tqdm(C_list):
C = 0.007
pred_prob = np.zeros((len(X), len(labels)))
pred_prob[:] = np.nan
for session in training_files.session.unique():
    mask = (training_files.session == session).values
    est = make_pipeline(
        StandardScaler(),
        LogisticRegression(
            C=C,
            max_iter=10**6,
        ),
    )
    est.fit(
        X[~mask],
        y[~mask],
    )
    pred_prob[mask] = est.predict_proba(X[mask])
assert not np.isnan(pred_prob).any()
pred = pred_prob.argmax(1)
display(
    pd.Series(
        {name: metric(y, pred_prob) for name, metric in metrics.items()},
        name="no_weight",
    ).round(3)
)
print()
display(
    pd.Series(
        {
            name: metric(y, pred_prob, sample_weight=session_weight)
            for name, metric in metrics.items()
        },
        name="session_weight",
    ).round(3)
)
print()
display(
    pd.Series(
        {
            name: metric(y, pred_prob, sample_weight=session_and_label_weight)
            for name, metric in metrics.items()
        },
        name="session_and_label_weight",
    ).round(3)
)

accuracy             0.524
balanced_accuracy    0.445
unweighted_f1        0.456
UAR                  0.445
logloss              1.259
Name: no_weight, dtype: float64

accuracy             0.526
balanced_accuracy    0.458
unweighted_f1        0.466
UAR                  0.458
logloss              1.206
Name: session_weight, dtype: float64

accuracy             0.458
balanced_accuracy    0.458
unweighted_f1        0.444
UAR                  0.458
logloss              1.432
Name: session_and_label_weight, dtype: float64